# MOQ

Goal of this page : return the quantities that need to be ordered by MOQ as required by the suppliers

Steps :
1. extract the product codes and quantities needed name of the dataframe is df in list to pandas, line 21
2. compare the quantities needed after process by the inventory (content df) vs the supplier's price list and adjust the quantities

In [337]:
import pandas as pd
import numpy as np
import math
import re
from pathlib import Path

In [342]:
dir_to_downloads = '/Users/Delphine/downloads'
p_downloads = Path(dir_to_downloads)

orders = pd.read_csv(p_downloads/'zapier-order-sheet - daily-orders.csv')



In [339]:
# sku_qt_processing(). This might have to change once we migrate to Shopify

def sku_qty_processing() :
    #upload and read file, get index size
    dir_to_downloads = '/Users/Delphine/downloads'
    p_downloads = Path(dir_to_downloads)

    orders = pd.read_csv(p_downloads/'zapier-order-sheet - daily-orders.csv')
    index_size = len(orders.index)

    #create new order file
    sku_qty = pd.DataFrame(columns=['Sku', 'Quantity', 'To Order'])

    # find the sku numbers and quantities
    i = 0
    j=0

    for i in range(0, index_size):
        text = orders['Text'][i]
        sku = re.findall('<p class="sku" style="margin-top: 0; margin-bottom: 0;">SKU: (.*)</p>', text)
        quantity = re.findall('<td class="item-qty" style="font-family: \'Open Sans\',\'Helvetica Neue\',Helvetica,Arial,sans-serif; vertical-align: top; text-align: center;">(.*)</td>', text)

        #add the skus and quantities to the sku_qty dataframe
        for j in range(0, len(sku)):
            sku_qty = sku_qty.append({'Sku':sku[j], 'Quantity':quantity[j]}, ignore_index=True)
            j+1

        i+1

    return sku_qty


In [343]:
sku_qty = sku_qty_processing()


sku_qty_size = len(sku_qty.index)
sku_qty.head()

,Sku,Quantity,To Order
0,71212.TK,1,NaN
1,978711,4,NaN
2,BW0745.TK,60,NaN
3,BW0735.TK,24,NaN
4,73600,1,NaN


In [298]:
dir_to_pricelist = '/Users/Delphine/Desktop/work/Price Lists'
p_pricelist = Path(dir_to_pricelist)

pricelist = pd.read_csv(p_pricelist/'All Suppliers Price List.csv')
pricelist = pricelist.drop(['Unnamed: 0'], axis=1)
pricelist.astype({'Item No.':'str'}).dtypes
pricelist.head()

,Item No.,Description,Range,MOQ,Tier 1 Price,Tier 2 Qty,Tier 2 Price,Tier 3 Qty,Tier 3 Price,Unit,Supplier
0,16051,COFFEE SPOON-18/10 JEWEL,NaN,12,$ 0.91,NaN,NaN,NaN,NaN,NaN,Trenton WA
1,16052,DESSERT FORK-18/10 JEWEL,NaN,12,$ 1.73,NaN,NaN,NaN,NaN,NaN,Trenton WA
2,16053,DESSERT SPOON-18/10 JEWEL,NaN,12,$ 1.73,NaN,NaN,NaN,NaN,NaN,Trenton WA
3,16054,SOUP SPOON-18/10 JEWEL,NaN,12,$ 1.73,NaN,NaN,NaN,NaN,NaN,Trenton WA
4,16055,TEASPOON-18/10 JEWEL,NaN,12,$ 1.06,NaN,NaN,NaN,NaN,NaN,Trenton WA


In [328]:

### YAY!!!!

for i in range (0, sku_qty_size):
    try:
        price_list_line = pricelist.loc[pricelist['Item No.'] == sku_qty['Sku'][i]]
        item = pricelist.loc[price_list_line.index]
        index_item = item.index[0]
        
        
    except:
        # The product code isn't on the price list, order the quantity originally ordered
        
        to_order1 = int(sku_qty['Quantity'][i]) 
        sku_qty.at[i, 'To Order'] = to_order1
        
    continue
    
    # Dealing with the cutlery having to be ordered in dozens only
   
    if ((price_list_line['Unit']  == 'Doz').all() | (price_list_line['Unit']  == 'DOZ').all()):


        moq2 = 12 
        needed2 = int(sku_qty['Quantity'][i])
        to_order2 = math.ceil(needed2 / moq2)
        sku_qty.at[i, 'To Order'] = to_order2


    else : 

    # The product code is on the price list, make the calculations based on MOQ
    # Most of products will follow this

        moq3 = int(item['MOQ'][index_item]) 
        needed3 = int(sku_qty['Quantity'][i])
        to_order3 = math.ceil(needed3 / moq3) * moq3
        sku_qty.at[i, 'To Order'] = to_order3

    


SyntaxError: invalid syntax (<ipython-input-328-1232140703df>, line 20)

In [325]:
price_list_line = pricelist.loc[pricelist['Item No.'] == sku_qty['Sku'][5]]
print(price_list_line)
if (price_list_line['Unit']  == 'Doz').all() | (price_list_line['Unit']  == 'DOZ').all():
    print(True)

     Item No.       Description   Range  MOQ Tier 1 Price  Tier 2 Qty  \
9747    13160  TABLE FORK-18/10  LONDON    1        22.00         1.0   

     Tier 2 Price  Tier 3 Qty Tier 3 Price Unit Supplier  
9747        22.00        15.0        19.20  Doz  Trenton  
True


In [344]:

### test

for i in range (0, sku_qty_size):
    #try:
    price_list_line = pricelist.loc[pricelist['Item No.'] == sku_qty['Sku'][i]]
    item = pricelist.loc[price_list_line.index]
    #index_item = item.index[0]
    qty_needed = int(sku_qty['Quantity'][i])
        
    #except:
        # The product code isn't on the price list, order the quantity originally ordered
     #   sku_qty.at[i, 'To Order'] = qty_needed
        
    #continue
    
    # Dealing with the cutlery having to be ordered in dozens only
    
    if ((price_list_line['Unit']  == 'Doz').all() | (price_list_line['Unit']  == 'DOZ').all()):
        
        to_order = math.ceil(qty_needed / 12)
        sku_qty.at[i, 'To Order'] = to_order
        
   
    else : 
                
    # The product code is on the price list, make the calculations based on MOQ
    # Most of products will follow this
        
        moq = int(item['MOQ'][item.index[0]]) 
        to_order = math.ceil(qty_needed / moq) * moq
        sku_qty.at[i, 'To Order'] = to_order
            
    
sku_qty

,Sku,Quantity,To Order
0,71212.TK,1,1
1,978711,4,4
2,BW0745.TK,60,60
3,BW0735.TK,24,24
4,73600,1,1
5,70907,1,1
6,05375.TK,1,1
7,ASOSIMR,1,1
8,926996,6,6
9,926096,6,6


In [312]:
price_list_line = pricelist.loc[pricelist['Item No.'] == sku_qty['Sku'][56]]
print(price_list_line.head())

print(item['MOQ']) ### the thing to fix
print(sku_qty['Quantity'][56])

item = pricelist.loc[price_list_line.index]
moq = int(item['MOQ'])
needed = int(sku_qty['Quantity'][56])
to_order = round(needed / moq) * moq
print(moq)
print(needed)
print(to_order)
#sku_qty.at[i, 'To Order'] = to_order

       Item No.                                    Description Range  MOQ  \
6594  BW0225.TK  BREW-SAGE/WHITE SAUCER TO SUIT BW0210/215/220  Brew    6   

     Tier 1 Price  Tier 2 Qty Tier 2 Price  Tier 3 Qty Tier 3 Price Unit  \
6594         2.27         NaN          NaN        36.0         2.06   EA   

     Supplier  
6594   Tomkin  
6594    6
Name: MOQ, dtype: int64
36
6
36
36


In [256]:
i=0

for i in range (0, sku_qty_size):
    price_list_line = pricelist.loc[pricelist['Item No.'] == sku_qty['Sku'][i]]
    item = pricelist.loc[price_list_line.index]
   
    # exception 1 : not in the price list = order the original quantity
    if sku_qty['Sku'][i] not in pricelist['Item No.']:
        to_order = int(sku_qty['Quantity'][i])
        sku_qty.at[i, 'To Order'] = to_order
    
    # exception 2 : the rest. If MOQ is 1 that doesn't make a difference
    if int(item['MOQ']) != int(sku_qty['Quantity'][i]):
        moq = int(item['MOQ'])
        needed = int(sku_qty['Quantity'][i])
        to_order = round(needed / moq) * moq
        sku_qty.at[i, 'To Order'] = to_order
    
    # exception 3 : cutlery
    #if (price_list_line['Unit']  == 'Doz').all or (price_list_line['Unit']  == 'DOZ').all:
    #    moq = 12 
    #    needed = int(sku_qty['Quantity'][i])
    #    to_order = round(needed / moq)
    #    sku_qty.at[i, 'To Order'] = to_order
        #= sku_qty.append({'To Order':to_order}, ignore_index=True)
    
    
            
            
    i+1
    
sku_qty

TypeError: cannot convert the series to <class 'int'>

In [185]:
price_list_line = pricelist.loc[pricelist['Item No.'] == sku_qty['Sku'][4]]
print(price_list_line)


print(item['MOQ']) ### the thing to fix
print(sku_qty['Quantity'][4])

item = pricelist.loc[price_list_line.index]
moq = int(item['MOQ'])
needed = int(sku_qty['Quantity'][4])
to_order = round(needed / moq) * moq
print('moq', moq)
print(needed)
print(to_order)
#sku_qty.at[i, 'To Order'] = to_order

      Item No.                     Description     Range  MOQ Tier 1 Price  \
9439  0398 106  MINI CHAMPAGNE FLUTE-177ml/6oz  TOSSWARE   48          1.1   
9477  0398 106  MINI CHAMPAGNE FLUTE-177ml/6oz  TOSSWARE   48         1.10   

      Tier 2 Qty Tier 2 Price  Tier 3 Qty Tier 3 Price  Unit Supplier  
9439         NaN          NaN        48.0          NaN  Each  Trenton  
9477         NaN         1.10        48.0         1.10  Each  Trenton  
6594    6
Name: MOQ, dtype: int64
44


TypeError: cannot convert the series to <class 'int'>

In [317]:
## SUR LA VOIE!

price_list_line_test = pricelist.loc[pricelist['Item No.'] == sku_qty['Sku'][1]]


item_test = pricelist.loc[price_list_line_test.index]
index_item_test = item_test.index[0]
index_item_test

14209

In [162]:
#all items

moq = item['MOQ'].astype(float)
needed = int(sku_qty['Quantity'][8])
to_order = round(needed / moq) * moq


it's to order 9439    0.0
9477    0.0
Name: MOQ, dtype: float64


In [200]:
# cutlery DOZ, Doz
price_list_line = pricelist.loc[pricelist['Item No.'] == sku_qty['Sku'][6]]
print(price_list_line['Unit'])

if (price_list_line['Unit']  == 'Doz').all or (price_list_line['Unit']  == 'DOZ').all:
    moq = 12 
    needed = int(sku_qty['Quantity'][6])
    to_order = round(needed / moq)
    print(to_order)

9867    Doz
Name: Unit, dtype: object
7


In [196]:
#brouillon


i = 0
ordered_items = pd.DataFrame()


for i in range (0, sku_qty_size):
    line = pricelist.loc[pricelist['Item No.'] == sku_qty['Sku'][i]]
    ordered_items.append(line)
    i + 1
    
ordered_items.head()

print(sku_qty['Sku'][1])
print(pricelist['Item No.'][1])
price_list_line = pricelist.loc[pricelist['Item No.'] == sku_qty['Sku'][1]]
print(line.index)
item = pricelist.loc[price_list_line.index]
print(item)
item['MOQ']
int(item['MOQ']) == int(sku_qty['Quantity'][1])

02750.TK
16052
Int64Index([11241], dtype='int64')
      Item No.                         Description      Range  MOQ  \
1471  02750.TK  CHEF INOX FRY BASKET-375x138x150mm  Chef Inox    1   

     Tier 1 Price  Tier 2 Qty Tier 2 Price  Tier 3 Qty Tier 3 Price Unit  \
1471        14.70         NaN          NaN         8.0        13.65   EA   

     Supplier  
1471   Tomkin  


False